# Calculate relevant variables

In [ ]:
import set_path

import numpy as np
import pandas as pd

import shapely.geometry as sg
import shapely.ops as so
import geopandas as gpd
from geopandas import GeoDataFrame
from centerline.geometry import Centerline

import networkx as nx
import momepy

from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

import upc_sw.poly_utils as poly_utils

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import pickle

In [ ]:
import warnings  # temporary, to supress deprecationwarnings from shapely
warnings.filterwarnings('ignore')

In [ ]:
#import importlib
#importlib.reload(poly_utils)

## Settings

In [ ]:
# BGT paths
#bgt_road_file = '../datasets/bgt/bgt_voetpad.gpkg' # old area east
#bgt_cl_file = '../datasets/output/bgt_cl_east.pkl'
bgt_road_file = '../datasets/output/amsterdam/bgt_voetpad.gpkg' 
bgt_cl_file = '../datasets/output/bgt_cl.pkl'

# A CRS tells Python how those coordinates relate to places on the Earth. Rijksdriehoek = epsg:28992
CRS = 'epsg:28992'

# Boundary for filtering out (in meters)
min_path_width = 0.4 

# Boundaries between the final colors green/orange/red (in meters)
width_lower = 0.9  
width_upper = 1.8
width_top = 2.9 

# Maximum distance between intended start point and start node (in meters)
max_dist = 3 

# Maximum length of linestring (in meters), otherwise cut
max_ls_length = 90

# Minimum interior size to remain in BGT data
min_interior_size = 10

# Min area size of a sidewalk polygon in sqm for which width will be computed
min_area_size = 5

# Resolution (in m) for min and avg width computation
width_resolution = 1

# Precision (in decimals) for min and avg width computation
width_precision = 1

In [ ]:
# Paths - old area east
#segments_file = '../datasets/output/sidewalk_segments.gpkg'
#output_file = '../datasets/output/final_output_segments_ae.geojson'
#output_image = '../datasets/output/final_output_image_ae.png'
#output_image_no = '../datasets/output/final_output_image_no_ae.png'

## Paths - entire city
segments_file1 = '../datasets/output/sidewalk_segments_1.gpkg'
segments_file2 = '../datasets/output/sidewalk_segments_2.gpkg'
segments_file3 = '../datasets/output/sidewalk_segments_3.gpkg'
segments_file4 = '../datasets/output/sidewalk_segments_4.gpkg'
segments_file5 = '../datasets/output/sidewalk_segments_5.gpkg'
segments_file6 = '../datasets/output/sidewalk_segments_6.gpkg'

output_file = '../datasets/output/final_output_segments_all.geojson'
output_image = '../datasets/output/final_output_image_all.png'
output_image_no = '../datasets/output/final_output_image_no_all.png'

## Prepare BGT data

In [ ]:
## Read BGT data
#df_bgt = gpd.read_file(bgt_road_file)
#df_bgt = df_bgt.set_crs(CRS, allow_override=True)
#df_bgt = df_bgt.drop_duplicates()

### Create centerlines without obstacles

In [ ]:
#df_bgt['geometry_no_holes'] = df_bgt.progress_apply(
#    lambda row: poly_utils.remove_interiors(row.geometry, min_interior_size), axis=1)
#df_bgt = df_bgt.set_geometry('geometry_no_holes')

In [ ]:
## Merge sidewalk polygons (optional, but should be in line with notebook 5 'merge_sidewalks' parameter)
#df_bgt = GeoDataFrame(geometry=gpd.GeoSeries(df_bgt['geometry_no_holes'].unary_union))
#df_bgt = gpd.GeoDataFrame(df_bgt.geometry.explode()) 

In [ ]:
## Ignore sidewalk polygons that are too small
#df_bgt['area'] = df_bgt['geometry'].area
#df_bgt = df_bgt[df_bgt.area > min_area_size]

In [ ]:
## Calculate centerlines
#df_bgt['centerlines'] = df_bgt.progress_apply(
#    lambda row: Centerline(row.geometry, interpolation_distance=0.5), axis=1)
#df_bgt = df_bgt.set_geometry('centerlines')

In [ ]:
#df_bgt['centerlines'] = df_bgt['centerlines'].progress_apply(so.linemerge)

In [ ]:
#df_bgt['centerlines'] = df_bgt['centerlines'].progress_apply(poly_utils.remove_short_lines)

### Store BGT dataframe with centerlines

In [ ]:
#with open(bgt_cl_file, 'wb') as f:
#    pickle.dump(df_bgt.to_dict(), f)

## Load prepared BGT data (if already present)

In [ ]:
df_bgt = GeoDataFrame(pd.read_pickle(bgt_cl_file, compression='infer'), 
                      crs=CRS, geometry='centerlines')

In [ ]:
df_bgt.head(2)

## Create relevant variable - full width

In [ ]:
# Get each centerline in separate row
df_bgt_exp = df_bgt.explode(ignore_index=True)

In [ ]:
# Remove few missing centerlines
df_bgt_exp = df_bgt_exp.dropna()
df_bgt_exp = df_bgt_exp.reset_index()

### Cut lines strings that are too long

In [ ]:
# remove geometry column and rename centerlines column to geometry for shorten linestrings function
df_bgt_cut = df_bgt_exp.drop('geometry', axis=1)
df_bgt_cut = df_bgt_cut.rename(columns = {'centerlines': 'geometry'})

In [ ]:
# Cut linestrings longer than a max length
df_bgt_cut['length'] = df_bgt_cut['geometry'].length
df_bgt_cut = poly_utils.shorten_linestrings(df_bgt_cut, max_ls_length, return_index=True)

In [ ]:
# Rename centerline column back again, to be able to merge properly
df_bgt_cut = df_bgt_cut.rename(columns = {'geometry': 'centerlines'})

In [ ]:
# Add shortened linestrings to previous dataframe
df_bgt_exp = df_bgt_exp.drop(['centerlines'], axis=1).merge(df_bgt_cut.drop(['area'], axis=1), 
                                                            how = 'outer', on = 'index')

In [ ]:
del df_bgt_cut

### Get width and color

In [ ]:
df_bgt_exp[['avg_width', 'min_width']] = df_bgt_exp.progress_apply(
    lambda row: poly_utils.get_avg_width_cl(row.geometry, row.centerlines, 
                                            width_resolution, width_precision), axis=1)

In [ ]:
conditions = [
    (df_bgt_exp['min_width'] < width_lower),
    (df_bgt_exp['min_width'] >= width_lower) & (df_bgt_exp['min_width'] < width_upper),
    (df_bgt_exp['min_width'] >= width_upper) & (df_bgt_exp['min_width'] < width_top),
    (df_bgt_exp['min_width'] >= width_top)
]

values_color = ['red', 'orange', 'lightgreen', 'green']
values_indication = ['<' + str(width_lower) + 'm', 
                     str(width_lower) + '-' + str(width_upper) + 'm', 
                     str(width_upper) + '-' + str(width_top) + 'm', 
                     '>' + str(width_top) + 'm']

In [ ]:
df_bgt_exp['full_width_color'] = np.select(conditions, values_color)
df_bgt_exp['full_width'] = np.select(conditions, values_indication)
df_bgt_exp['full_width'].value_counts()

## Import point cloud and centerline data

In [ ]:
# Read lines with widths (calculated in notebook 5)
#df_segments = gpd.read_file(segments_file, crs=CRS)
df_segments1 = gpd.read_file(segments_file1, crs=CRS)
df_segments2 = gpd.read_file(segments_file2, crs=CRS)
df_segments3 = gpd.read_file(segments_file3, crs=CRS)
df_segments4 = gpd.read_file(segments_file4, crs=CRS)
df_segments5 = gpd.read_file(segments_file5, crs=CRS)
df_segments6 = gpd.read_file(segments_file6, crs=CRS)

In [ ]:
# Add together
df_segments = pd.concat([df_segments1, df_segments2, df_segments3, 
                         df_segments4, df_segments5, df_segments6]).reset_index()

In [ ]:
# Remove dataframes
del df_segments1
del df_segments2
del df_segments3
del df_segments4
del df_segments5
del df_segments6

## Process data

### Remove too narrow paths

In [ ]:
# Apply minimal path width
df_segments_wide = df_segments[df_segments['min_width'] > min_path_width].reset_index(drop=True)
print(df_segments.shape)
print(df_segments_wide.shape)

In [ ]:
# Remove short lines
mls_per_id = poly_utils.create_mls_per_sidewalk(df_segments_wide, crs=CRS)
mls_per_id['geometry'] = mls_per_id['geometry'].progress_apply(
                        lambda x: poly_utils.remove_short_lines(x))

In [ ]:
# Apply selection of longer lines to original dataframe
long_segments_df = gpd.GeoDataFrame(mls_per_id.geometry.explode())
df_segments_wide = df_segments_wide.merge(long_segments_df, how='inner')
df_segments_wide.shape

### Use color codes

In [ ]:
conditions = [
    (df_segments_wide['min_width'] < width_lower),
    (df_segments_wide['min_width'] >= width_lower) & (df_segments_wide['min_width'] < width_upper),
    (df_segments_wide['min_width'] >= width_upper) & (df_segments_wide['min_width'] < width_top),
    (df_segments_wide['min_width'] >= width_top)
]

values = ['red', 'orange', 'lightgreen', 'green']

In [ ]:
df_segments_wide['min_width_color'] = np.select(conditions, values)
df_segments_wide['min_width_color'].value_counts()

In [ ]:
# Add width factor, for calculating the weights of the paths later
values = [1000000000, 1000000, 1000, 1]
df_segments_wide['min_width_factor'] = np.select(conditions, values).astype('int64')

### Take point cloud coverage into account

In [ ]:
df_segments_wide['min_width_factor'][df_segments_wide['pc_coverage'] == False] = 1000000000001

In [ ]:
df_segments_wide['min_width_factor'].value_counts()

## Create relevant variable - obstacle-free width

In [ ]:
# Create final dataframe
final_df = pd.DataFrame()

for i in range(len(df_bgt['centerlines'])):
    
    # Get centerline
    my_centerline = df_bgt['centerlines'].values[i]
    
    # Create dataframe with linestrings of centerline
    centerline_df = poly_utils.create_df_centerlines(my_centerline)
    
    # Cut linestrings that are too long
    centerline_df = poly_utils.shorten_linestrings(centerline_df, max_ls_length)
        
    # Get sidewalk polygon for this centerline
    my_sidewalk = df_bgt['geometry'].values[i]
    
    # Create graph for all paths withing this sidewalk polygon
    df_sidewalk = df_segments_wide[df_segments_wide['geometry'].within(my_sidewalk)].reset_index(drop=True)
    G = momepy.gdf_to_nx(df_sidewalk, approach="primal", multigraph=True)
    
    for j in range(len(centerline_df['geometry'])):
        
        # Get line
        my_line = centerline_df.iloc[[j]]['geometry'].values[0]
        
        if len(list(G.nodes)) > 0:
            # Check if my_line has start and end (not a ring)
            if len(my_line.boundary) == 0:
                print('no route calculated for line (j)', j, 'in sidewalk (i)', i, '(ring)')
                centerline_df['route_weight'][j] = 0
            else:
                # Get origin and destination location
                origin_point, dest_point = my_line.boundary
                origin_node_loc = so.nearest_points(origin_point, sg.MultiPoint(list(G.nodes)))[1]
                dest_node_loc = so.nearest_points(dest_point, sg.MultiPoint(list(G.nodes)))[1]

                # Get origin and destination node
                origin_node = (origin_node_loc.x, origin_node_loc.y)
                dest_node = (dest_node_loc.x, dest_node_loc.y)   

                # Get weight of optimal route in graph
                try:
                    route_weight = nx.shortest_path_length(G, origin_node, dest_node,
                                                           weight='min_width_factor')
                    if (origin_point.distance(origin_node_loc) < max_dist) and (dest_point.distance(dest_node_loc) < max_dist):
                        centerline_df['route_weight'][j] = route_weight
                    else:
                        print('origin and/or destination node too far from line start/end for line (j)', j, 'in sidewalk (i)', i) 
                        centerline_df['route_weight'][j] = np.nan
                except nx.NetworkXNoPath:
                    print('no route found for line (j)', j, 'in sidewalk (i)', i)
                    centerline_df['route_weight'][j] = 1000000000000
        else:
            print('network has zero nodes')

    # Append data to final dataframe
    final_df = final_df.append(centerline_df)
final_df = final_df.reset_index()

In [ ]:
# Get final color of the routes
final_df['final_color'] = final_df.progress_apply(
    lambda row: poly_utils.get_route_color(row.route_weight), axis=1)

In [ ]:
final_df['final_color'].value_counts()

In [ ]:
final_df_group = final_df.groupby('final_color').sum()
final_df_group['length_perc'] = round(100 * final_df_group['length'] / sum(final_df_group['length']), 1)
final_df_group[['length_perc']]

## Post-process obstacle-free width

In [ ]:
# Get dataframe with only valid lines
final_df_select = final_df[final_df['final_color'].isin(['green', 'lightgreen', 'orange', 
                                                         'red', 'purple', 'black', 'grey'])]
final_df_select = final_df_select.reset_index()

In [ ]:
# Recolor paths with unknown widths
final_df_select['final_color'] = final_df_select['final_color'].replace(['black', 'purple'], 'grey')

In [ ]:
# Add meter indication
conditions = [
    (final_df_select['final_color'] == 'red'),
    (final_df_select['final_color'] == 'orange'),   
    (final_df_select['final_color'] == 'lightgreen'),
    (final_df_select['final_color'] == 'green'),  
    (final_df_select['final_color'] == 'grey')
]
values_indication = ['<' + str(width_lower) + 'm', 
                     str(width_lower) + '-' + str(width_upper) + 'm', 
                     str(width_upper) + '-' + str(width_top) + 'm', 
                     '>' + str(width_top) + 'm',
                    'unknown']
final_df_select['obstacle_free_width'] = np.select(conditions, values_indication)

In [ ]:
# Select only relevant columns
final_df_select = final_df_select[['geometry', 'final_color', 'obstacle_free_width']]

In [ ]:
# Add object ID
final_df_select['object_id'] = final_df_select.index

## Post-process full width

In [ ]:
# Apply minimal path width on BGT centerlines
print(df_bgt_exp.shape)
df_bgt_exp = df_bgt_exp[df_bgt_exp['min_width'] > min_path_width].reset_index(drop=True)
print(df_bgt_exp.shape)

In [ ]:
# Prepare BGT dataframe for merging
df_bgt_exp = df_bgt_exp[['centerlines', 'full_width', 'full_width_color']]
df_bgt_exp = df_bgt_exp.rename(columns = {'centerlines': 'geometry'})

## Merge relevant variables

In [ ]:
# Merge
final_df_merge = final_df_select.merge(df_bgt_exp, how ='inner')  

In [ ]:
final_df_merge.head()

In [ ]:
### TO DO - Remove invalid obstacle free width indications

## Store final output

In [ ]:
# Adjust crs: maps.amsterdam.nl requested WGS84 (Lng-Lat) 
final_df_merge = final_df_merge.set_crs(CRS) 
df_projected = final_df_merge.to_crs('epsg:4326') 

In [ ]:
with open(output_file, 'w') as f:
    f.write(df_projected.to_json())

## Visualizations

In [ ]:
# Boundaries for plotting a subset of the data
x_min = 122400 # old area east
x_max = 123600
y_min = 485400
y_max = 486000

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(10,5), dpi=2000, 
                                    frameon=False, constrained_layout=True)

# Plot process step 1
df_bgt.set_geometry('geometry').plot(ax=ax1, color="silver")
df_segments_wide.plot(ax=ax1, linewidth=0.1, color=df_segments_wide['min_width_color'])
ax1.axis('off')

# Plot process step 2
df_bgt.set_geometry('geometry').plot(ax=ax2, color="silver")
final_df.plot(ax=ax2, linewidth=0.5, color=final_df.final_color)  
ax2.axis('off')

# Plot process step 3
df_bgt.set_geometry('geometry').plot(ax=ax3, color="silver")
final_df_select.plot(ax=ax3, linewidth=0.5, color=final_df_select.final_color)  
ax3.axis('off')

# Create legends
wide = mpatches.Patch(color='green', label='>' + str(width_top) + 'm')
acc = mpatches.Patch(color='lightgreen', label= str(width_upper) + '-' + str(width_top) + 'm')
narrow = mpatches.Patch(color='orange', label= str(width_lower) + '-' + str(width_upper) + 'm')
notacc = mpatches.Patch(color='red', label='<' + str(width_lower) + 'm')
rno = mpatches.Patch(color='darkred', label='no route possible')
rnan = mpatches.Patch(color='purple', label='node too far')
rl0 = mpatches.Patch(color='black', label='route length 0')
rnpc = mpatches.Patch(color='grey', label='no pc coverage')
runk = mpatches.Patch(color='grey', label='unknown')

legend1 = plt.legend(handles=[wide,acc,narrow,notacc,rno,rnan,rl0,rnpc], 
           bbox_to_anchor=(-0.5, -0.5, 0.5, 0.5))
plt.legend(handles=[wide,acc,narrow,notacc,runk], 
           bbox_to_anchor=(0.5, -0.5, 0.5, 0.5))
plt.gca().add_artist(legend1)

# Set plot limits
#ax1.set_xlim([x_min, x_max])   
#ax1.set_ylim([y_min, y_max])
#ax2.set_xlim([x_min, x_max])
#ax2.set_ylim([y_min, y_max])
#ax3.set_xlim([x_min, x_max])
#ax3.set_ylim([y_min, y_max])

plt.savefig(output_image, bbox_inches='tight')
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5), dpi=2000, 
                                    frameon=False, constrained_layout=True)

# Plot process step 3
df_bgt.set_geometry('geometry').plot(ax=ax1, color="silver")
final_df_merge.plot(ax=ax1, linewidth=0.5, color=final_df_merge.final_color)  
ax1.axis('off')

# Plot process step 3
df_bgt.set_geometry('geometry').plot(ax=ax2, color="silver")
final_df_merge.plot(ax=ax2, linewidth=0.5, color=final_df_merge.full_width_color)  
ax2.axis('off')

# Create legends
wide = mpatches.Patch(color='green', label='>' + str(width_top) + 'm')
acc = mpatches.Patch(color='lightgreen', label= str(width_upper) + '-' + str(width_top) + 'm')
narrow = mpatches.Patch(color='orange', label= str(width_lower) + '-' + str(width_upper) + 'm')
notacc = mpatches.Patch(color='red', label='<' + str(width_lower) + 'm')
runk = mpatches.Patch(color='grey', label='unknown')

legend1 = plt.legend(handles=[wide,acc,narrow,notacc,runk], 
           bbox_to_anchor=(-0.5, -0.5, 0.5, 0.5), title = "obstacle free width")
plt.legend(handles=[wide,acc,narrow,notacc], 
           bbox_to_anchor=(0.5, -0.5, 0.5, 0.5), title = "full width")
plt.gca().add_artist(legend1)

# Set plot limits
#ax1.set_xlim([x_min, x_max])   
#ax1.set_ylim([y_min, y_max])
#ax2.set_xlim([x_min, x_max])   
#ax2.set_ylim([y_min, y_max])

plt.savefig(output_image_no, bbox_inches='tight')

#plt.show()